<a href="https://colab.research.google.com/github/rajilsaj/FICOchallenge/blob/web-app/WebApp/webApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [127]:
!pip install -q streamlit

In [128]:
!pkill streamlit
!streamlit cache clear
!pip install pyngrok -q

In [129]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦
up to date, audited 23 packages in 890ms
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠦

In [130]:
# ============================================
# DRIVE + LOGO MANAGEMENT CELL (RUN FIRST)
# ============================================

from google.colab import drive
import os
import shutil

# 1️⃣ Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# 2️⃣ Define paths
DRIVE_LOGO_PATH = "/content/drive/MyDrive/FICO Analytic Challenge/WebApp/assets/FICO2025-Full.jpg"
LOCAL_ASSET_DIR = "/content/webapp_assets"
LOCAL_LOGO_PATH = f"{LOCAL_ASSET_DIR}/FICO2025-Full.jpg"

# 3️⃣ Create local asset directory
os.makedirs(LOCAL_ASSET_DIR, exist_ok=True)

# 4️⃣ Check if image exists in Drive
if os.path.exists(DRIVE_LOGO_PATH):
    print("✅ Logo found in Google Drive.")

    # 5️⃣ Copy locally for better performance
    shutil.copy(DRIVE_LOGO_PATH, LOCAL_LOGO_PATH)
    print("✅ Logo copied locally for Streamlit.")
else:
    print("❌ Logo not found. Check folder path carefully.")

# 6️⃣ Final check
if os.path.exists(LOCAL_LOGO_PATH):
    print("🚀 Logo ready at:", LOCAL_LOGO_PATH)
else:
    print("⚠️ Local logo missing.")

Mounted at /content/drive
✅ Logo found in Google Drive.
✅ Logo copied locally for Streamlit.
🚀 Logo ready at: /content/webapp_assets/FICO2025-Full.jpg


In [136]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import time
import os

# ==================================================
# PAGE CONFIG
# ==================================================
st.set_page_config(
    page_title="FICO Synthetic Intelligence Platform",
    page_icon="🔵",
    layout="wide"
)

LOGO_PATH = "/content/webapp_assets/FICO2025-Full.jpg"

# ==================================================
# SESSION STATE
# ==================================================
if "page" not in st.session_state:
    st.session_state.page = "Dashboard"

if "generated_df" not in st.session_state:
    st.session_state.generated_df = None

# ==================================================
# SIDEBAR (LOGO FIXED ON TOP)
# ==================================================
with st.sidebar:
    if os.path.exists(LOGO_PATH):
        st.image(LOGO_PATH, use_container_width=True)
    st.markdown("---")

# ==================================================
# TOP NAVIGATION
# ==================================================
col1, col2, col3, col4, col5, col6, col7 = st.columns(7)

if col1.button("Dashboard"):
    st.session_state.page = "Dashboard"

if col2.button("Generation"):
    st.session_state.page = "Generation"

if col3.button("Test Set"):
    st.session_state.page = "Test Set"

if col4.button("Training"):
    st.session_state.page = "Training"

if col5.button("Fine-Tuning"):
    st.session_state.page = "Fine-Tuning"

if col6.button("Reports"):
    st.session_state.page = "Reports"

if col7.button("Projects"):
    st.session_state.page = "Projects"

st.markdown("---")

page = st.session_state.page

# ==================================================
# DASHBOARD
# ==================================================
if page == "Dashboard":

    st.title("Dashboard")

    col1, col2, col3 = st.columns(3)
    col1.metric("Synthetic Rows",
                0 if st.session_state.generated_df is None else len(st.session_state.generated_df))
    col2.metric("Active Model", "Llama 3.1")
    col3.metric("System Status", "Operational")


# ==================================================
# GENERATION
# ==================================================
elif page == "Generation":

    st.title("Synthetic Data Generation")

    # -----------------------------------------
    # SIDEBAR CONTROLS
    # -----------------------------------------
    with st.sidebar:

        st.header("Upload Existing Dataset (Optional)")
        uploaded = st.file_uploader("Upload CSV", type=["csv"])

        st.markdown("---")
        st.markdown("### OR")
        st.markdown("---")

        st.header("Generate Custom Dataset")

        model = st.selectbox(
            "Model",
            ["Llama 3.1 (Unsloth)", "Mistral", "Ollama"]
        )

        temperature = st.slider("Temperature", 0.1, 1.5, 0.7)

        num_scenarios = st.number_input(
            "Number of Scenarios",
            min_value=1,
            max_value=100,
            value=5
        )

        num_variants = st.number_input(
            "Variants per Scenario",
            min_value=1,
            max_value=20,
            value=5
        )

        sentiments = st.multiselect(
            "Sentiments",
            ["angry", "confused", "neutral", "frustrated"],
            default=["neutral"]
        )

        speech = st.multiselect(
            "Speech Types",
            ["casual", "professional", "slang", "typos"],
            default=["professional"]
        )

        generate_btn = st.button("Generate Dataset", use_container_width=True)

    # -----------------------------------------
    # MAIN CONTENT
    # -----------------------------------------

    if uploaded:
        with st.status("Uploading dataset...", expanded=False) as status:
            time.sleep(1)
            df = pd.read_csv(uploaded)
            st.session_state.generated_df = df
            status.update(label="Upload complete ✅", state="complete")

        st.toast("Dataset uploaded successfully!")

    if generate_btn:
        with st.status("Generating synthetic dataset...", expanded=True) as status:
            time.sleep(1)
            status.write("Initializing model...")
            time.sleep(1)
            status.write("Applying temperature and parameters...")
            time.sleep(1)

            df = pd.DataFrame({
                "intent": ["PAYMENT"] * num_scenarios,
                "scenario": [
                    f"Scenario {i+1} using {model}"
                    for i in range(num_scenarios)
                ],
                "temperature": temperature,
                "variants": num_variants
            })

            st.session_state.generated_df = df
            status.update(label="Generation complete ✅", state="complete")

        st.toast("Synthetic dataset generated!")

    if st.session_state.generated_df is not None:
        st.subheader("Preview")
        st.dataframe(st.session_state.generated_df.head())

        csv = st.session_state.generated_df.to_csv(index=False).encode("utf-8")

        st.download_button(
            "Download Dataset",
            csv,
            "synthetic_dataset.csv",
            "text/csv"
        )


# ==================================================
# TEST SET
# ==================================================
elif page == "Test Set":

    st.title("Model Evaluation")

    with st.sidebar:
        test_file = st.file_uploader("Upload Test Set", type=["csv"])
        evaluate_btn = st.button("Run Evaluation", use_container_width=True)

    if test_file:
        df_test = pd.read_csv(test_file)
        st.dataframe(df_test.head())

    if evaluate_btn:
        with st.status("Running evaluation...", expanded=True) as status:
            time.sleep(1)
            status.write("Loading model...")
            time.sleep(1)
            status.write("Computing metrics...")
            time.sleep(1)
            status.update(label="Evaluation complete ✅", state="complete")

        st.metric("Accuracy", "89%")
        st.metric("F1 Score", "0.87")


# ==================================================
# TRAINING
# ==================================================
elif page == "Training":

    st.title("Model Training")

    with st.sidebar:
        epochs = st.slider("Epochs", 1, 10, 3)
        train_btn = st.button("Start Training", use_container_width=True)

    if train_btn:
        with st.status("Training model...", expanded=True) as status:
            for i in range(epochs):
                time.sleep(0.7)
                status.write(f"Epoch {i+1}/{epochs} complete")
            status.update(label="Training finished ✅", state="complete")

        st.toast("Training successful!")


# ==================================================
# FINE-TUNING
# ==================================================
elif page == "Fine-Tuning":

    st.title("Fine-Tuning")

    with st.sidebar:
        lr = st.slider("Learning Rate", 0.00001, 0.01, 0.001)
        fine_btn = st.button("Apply Fine-Tuning", use_container_width=True)

    if fine_btn:
        with st.status("Applying fine-tuning...", expanded=True) as status:
            time.sleep(2)
            status.update(label="Fine-tuning complete ✅", state="complete")

        st.toast("Fine-tuning applied.")


# ==================================================
# REPORTS
# ==================================================
elif page == "Reports":

    st.title("Analytics & Reports")

    chart_data = pd.DataFrame(
        np.random.randn(20, 3),
        columns=["Precision", "Recall", "F1"]
    )

    st.line_chart(chart_data)


# ==================================================
# PROJECTS
# ==================================================
elif page == "Projects":

    st.title("Projects")

    st.write("Current experiments and previous runs will appear here.")

Overwriting app.py


In [132]:
!pip install pyngrok -q

from pyngrok import ngrok
import time
import os

# Replace YOUR_TOKEN_HERE with your real token
ngrok.set_auth_token("2vfqmZ0LaVtY2NIjJN4RiesWlhV_BVLTZiUFNjhF2CFRDTLM")

In [133]:
# ==========================================
# BULLETPROOF STREAMLIT + NGROK LAUNCH
# ==========================================

import subprocess
import time
from pyngrok import ngrok

# Kill any existing processes
!pkill -f streamlit
ngrok.kill()

# Start Streamlit cleanly
process = subprocess.Popen(
    ["streamlit", "run", "app.py",
     "--server.port", "8501",
     "--server.headless", "true",
     "--browser.gatherUsageStats", "false"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.STDOUT
)

# Wait for server to fully boot
time.sleep(6)

# Create secure tunnel
public_url = ngrok.connect(8501, bind_tls=True)

print("====================================")
print("🚀 OPEN YOUR APP HERE:")
print(public_url.public_url)
print("====================================")

🚀 OPEN YOUR APP HERE:
https://79a5-34-83-133-127.ngrok-free.app


In [134]:
#z!npx localtunnel --port 8501
!tail -n 50 streamlit.log


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.83.133.127:8501

  Stopping...
